In [78]:
!pip install rdflib

In [79]:
import pandas as pd
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import Namespace, NamespaceManager
from rdflib.plugins import sparql
from rdflib.namespace import RDF, RDFS, XSD
from rdflib.serializer import Serializer

pd.set_option('display.max_columns', None)

In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
# Create an emprty graph
g = Graph()

g.parse('/content/drive/MyDrive/graphs/music_ontology.owl')

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("It better be!")

# Print the number of "triples" in the Graph
print(f"Graph g has {len(g)} statements.")

Graph g has 111 statements.


In [82]:
# Classes
Track_class = URIRef("http://www.semanticweb.org/music_ontology#Track")
Performer_class = URIRef("http://www.semanticweb.org/music_ontology#Performer")
Genre_class = URIRef("http://www.semanticweb.org/music_ontology#Genre")
Album_class = URIRef("http://www.semanticweb.org/music_ontology#Album")

#object properties
hasAssociatedGenre_op = URIRef("http://www.semanticweb.org/music_ontology#hasAssociatedGenre")
hasGenre_op = URIRef("http://www.semanticweb.org/music_ontology#hasGenre")
partOf_op = URIRef("http://www.semanticweb.org/music_ontology#partOf")
performedBy_op = URIRef("http://www.semanticweb.org/music_ontology#performedBy")

#data properties
acousticness_dp = URIRef("http://www.semanticweb.org/music_ontology#acousticness")
country_dp = URIRef("http://www.semanticweb.org/music_ontology#country")
danceability_dp = URIRef("http://www.semanticweb.org/music_ontology#danceability")
duration_dp = URIRef("http://www.semanticweb.org/music_ontology#duration")
energy_dp = URIRef("http://www.semanticweb.org/music_ontology#energy")
instrumentalness_dp = URIRef("http://www.semanticweb.org/music_ontology#instrumentalness")
label_dp = URIRef("http://www.semanticweb.org/music_ontology#label")
liveness_dp = URIRef("http://www.semanticweb.org/music_ontology#liveness")
loudness_dp = URIRef("http://www.semanticweb.org/music_ontology#loudness")
name_dp = URIRef("http://www.semanticweb.org/music_ontology#name")
popularity_dp = URIRef("http://www.semanticweb.org/music_ontology#popularity")
releaseYear_dp = URIRef("http://www.semanticweb.org/music_ontology#releaseYear")
speechiness_dp = URIRef("http://www.semanticweb.org/music_ontology#speechiness")
tempo_dp = URIRef("http://www.semanticweb.org/music_ontology#tempo")
tracksTotal_dp = URIRef("http://www.semanticweb.org/music_ontology#tracksTotal")
valence_dp = URIRef("http://www.semanticweb.org/music_ontology#valence")

### Prep_data

In [83]:
df = pd.read_csv('/content/drive/MyDrive/graphs/final_df.csv')
albums = pd.read_csv('/content/drive/MyDrive/graphs/albums.csv')
artist = pd.read_csv('/content/drive/MyDrive/graphs/artist.csv')
tracks = pd.read_csv('/content/drive/MyDrive/graphs/tracks.csv')
performers = pd.read_csv('/content/drive/MyDrive/graphs/performers.csv')

In [84]:
albums.head()

,album_type,available_markets,external_urls,href,id,images,name,release_date,release_date_precision,total_tracks,type,uri,performer_id
0,album,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",{'spotify': 'https://open.spotify.com/album/53...,https://api.spotify.com/v1/albums/53NpZCeQB2Jd...,53NpZCeQB2JdALLsHsObjg,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Biokinetics,2012-02-17,day,8,album,spotify:album:53NpZCeQB2JdALLsHsObjg,e63f1350-8e5e-43c3-b32f-a224f9a9f778
1,album,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",{'spotify': 'https://open.spotify.com/album/31...,https://api.spotify.com/v1/albums/31m3CasBlRFc...,31m3CasBlRFcf8exTSkM1r,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",No Protection,1995-02-17,day,8,album,spotify:album:31m3CasBlRFcf8exTSkM1r,717866a0-7fdc-458f-9ad5-557a4bcbc27b
2,album,[],{'spotify': 'https://open.spotify.com/album/7C...,https://api.spotify.com/v1/albums/7CTzBa9yMtj5...,7CTzBa9yMtj5dnzkdlRdYa,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Nighttime Birds,1997,year,9,album,spotify:album:7CTzBa9yMtj5dnzkdlRdYa,7e22cc61-630e-4204-a759-8ff90ac6e3b7
3,compilation,[],{'spotify': 'https://open.spotify.com/album/2V...,https://api.spotify.com/v1/albums/2ViLIuBfZVx8...,2ViLIuBfZVx8ijQ7bQwhaw,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Aftershock - The Island Years 1985 - 1990,2013-01-01,day,54,album,spotify:album:2ViLIuBfZVx8ijQ7bQwhaw,bcc31f55-036b-4c47-8636-ab52f791922f
4,album,"['AG', 'AR', 'BB', 'BO', 'BR', 'BS', 'BZ', 'CL...",{'spotify': 'https://open.spotify.com/album/4L...,https://api.spotify.com/v1/albums/4LOqtdV9dCY7...,4LOqtdV9dCY7NX7iNK1lBe,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Unwritten,2004-09-06,day,13,album,spotify:album:4LOqtdV9dCY7NX7iNK1lBe,bc3de894-8a3f-47d7-90ff-23dfdabb5b15


In [85]:
artist.head()

,external_urls,href,id,name,type,uri
0,{'spotify': 'https://open.spotify.com/artist/5...,https://api.spotify.com/v1/artists/5bQ4kqxuAV7...,5bQ4kqxuAV7tBI7dSs3sB1,Xmal Deutschland,artist,spotify:artist:5bQ4kqxuAV7tBI7dSs3sB1
1,{'spotify': 'https://open.spotify.com/artist/3...,https://api.spotify.com/v1/artists/3p4wAteGgJE...,3p4wAteGgJEu6L5eMYJ2dW,Sea Of Treachery,artist,spotify:artist:3p4wAteGgJEu6L5eMYJ2dW
2,{'spotify': 'https://open.spotify.com/artist/0...,https://api.spotify.com/v1/artists/0kEfub5RzlZ...,0kEfub5RzlZOB2zGomqVSU,Gil Scott-Heron,artist,spotify:artist:0kEfub5RzlZOB2zGomqVSU
3,{'spotify': 'https://open.spotify.com/artist/7...,https://api.spotify.com/v1/artists/7zW6wPmVLv4...,7zW6wPmVLv4JDJM271gbFc,Wim Mertens,artist,spotify:artist:7zW6wPmVLv4JDJM271gbFc
4,{'spotify': 'https://open.spotify.com/artist/6...,https://api.spotify.com/v1/artists/64d1rUxfizS...,64d1rUxfizSAOE9UbMnUZd,Sezen Aksu,artist,spotify:artist:64d1rUxfizSAOE9UbMnUZd


In [86]:
tracks.head()

,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri,album_id,performer_id
0,['FR'],1,256253,False,{'isrc': 'GBUM70601405'},{'spotify': 'https://open.spotify.com/track/1U...,https://api.spotify.com/v1/tracks/1URvfSQ1ZEqd...,1URvfSQ1ZEqdSjil0Gfcuf,False,I Talk Too Much,0,NaN,6,track,spotify:track:1URvfSQ1ZEqdSjil0Gfcuf,7pgzmZ2rw2sLH6meC3EMtz,ac1f43b2-d974-4fa9-a0cb-d0fbbaacbd2a
1,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,383493,False,{'isrc': 'USCA28200152'},{'spotify': 'https://open.spotify.com/track/2u...,https://api.spotify.com/v1/tracks/2uScQp1u1z9t...,2uScQp1u1z9tKCHEpUuj4M,False,Dark Avenger,28,NaN,6,track,spotify:track:2uScQp1u1z9tKCHEpUuj4M,1WeQxVMtBkCqVal9AOQC5A,235506f5-d253-4b30-a6eb-fb78e16a9be1
2,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,224493,False,{'isrc': 'GBF080201255'},{'spotify': 'https://open.spotify.com/track/1q...,https://api.spotify.com/v1/tracks/1q1l9qgvNZZK...,1q1l9qgvNZZKoBJtG03q1P,False,Hill Farmer's Blues,40,NaN,3,track,spotify:track:1q1l9qgvNZZKoBJtG03q1P,1hHn4rCodVpbrO76BunvYx,73ca7c79-fce1-4189-b7e2-a504f606d6ae
3,[],1,167920,False,{'isrc': 'USKRS1054101'},{'spotify': 'https://open.spotify.com/track/1o...,https://api.spotify.com/v1/tracks/1ojlyZZJJsic...,1ojlyZZJJsicC0T3XtBJTn,False,Ballad of Big Nothing,0,NaN,1,track,spotify:track:1ojlyZZJJsicC0T3XtBJTn,150GIjws2PSxafQI1vCGhj,9a7cd374-a66f-4c12-9a19-6ba3ebbbf6f5
4,[],1,430466,False,{'isrc': 'DEPD61100417'},{'spotify': 'https://open.spotify.com/track/0u...,https://api.spotify.com/v1/tracks/0utlk2QtBSV0...,0utlk2QtBSV0s4FgBuNV9v,False,Taxi,0,NaN,2,track,spotify:track:0utlk2QtBSV0s4FgBuNV9v,1Vz38SPJRFRTZhsL7HZ3fd,d71f7426-f83c-46ae-a3cc-3e280221cf7d


In [87]:
performers.head()

,performer_id,artist_ids
0,44479cef-0056-43ca-a3f5-7ee5aa661e61,['6dD5LzTwdYS3gRvRSCOIVP']
1,04bb58fe-572f-4e87-9e7f-760093e6a218,['4KlYg0F5KG9QNDFKaeTNAy']
2,34292b88-d43f-4b24-b088-193901f0dbbb,['3uUghlgJJQPZOHB6KRrSXZ']
3,7a015f39-70ef-447e-a625-dbcfdf05e958,['1uS4cYRRZRtauASIM9RxBG']
4,b67a5c99-d51d-4083-ae60-968936eb3488,['01GliotOQOW6gTjIIUEhNR']


In [88]:
df.head()

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,user_id,playcount,song_frequency_per_user,rating
0,TRHNGJL128F9310B69,East St. Louis Toodle-Oo,Steely Dan,https://p.scdn.co/mp3-preview/d5e264edddf69878...,0M5U3BpuVd2MscKJnytbFu,"jazz, classic_rock, instrumental, progressive_...",Rock,1985,169453,0.654,0.283,0,-13.873,0,0.0372,0.771000,0.153000,0.3290,0.641,153.745,4,ffffcfeb0c1b66bd212ea58d918c7dc62fb9c3a5,1.0,0.012821,1.0
1,TRRXFHO128EF3550BC,Fast as I Can,Stone Temple Pilots,https://p.scdn.co/mp3-preview/d3c68ac4bdb4cc91...,1MjCvnob720XbcxplOpaL6,"alternative_rock, grunge",NaN,2010,213586,0.481,0.965,0,-4.755,1,0.0452,0.000194,0.000002,0.4540,0.659,97.445,4,ffffcfeb0c1b66bd212ea58d918c7dc62fb9c3a5,1.0,0.012821,1.0
2,TRXVLQC128F9324EFB,Ray Gun,The Bird and the Bee,https://p.scdn.co/mp3-preview/ce0657623a9efa60...,1HiI9wIBqVNCnjw0aIL6jG,"alternative, indie, psychedelic, trip_hop, dow...",NaN,2008,281440,0.498,0.515,0,-9.780,0,0.0298,0.596000,0.000148,0.1220,0.358,169.924,4,ffffcfeb0c1b66bd212ea58d918c7dc62fb9c3a5,4.0,0.051282,1.0
3,TRHSGXW12903CE895F,Drunken Lullabies,Flogging Molly,https://p.scdn.co/mp3-preview/8d3029a482fa122b...,0aE8nMLrrd2m6lTscFIbZw,"rock, folk, punk, punk_rock",NaN,2002,230640,0.385,0.969,7,-4.234,1,0.0748,0.001500,0.046200,0.0817,0.854,156.177,4,ffffcfeb0c1b66bd212ea58d918c7dc62fb9c3a5,1.0,0.012821,1.0
4,TRLHRQJ128F424831C,Denia,Manu Chao,https://p.scdn.co/mp3-preview/e7b2312bbf6ba03a...,0aQ5WLFnSpqEeTo2LCQ4ja,reggae,Latin,2001,279480,0.657,0.835,4,-6.106,0,0.0327,0.346000,0.000341,0.7040,0.731,120.015,4,ffffcfeb0c1b66bd212ea58d918c7dc62fb9c3a5,8.0,0.102564,1.5


In [89]:
df.drop_duplicates(subset=['track_id'], inplace=True)

In [90]:
data = df.merge(tracks, how='inner', left_on='spotify_id', right_on='id')
data = data.merge(artist, how='inner', left_on='artist', right_on='name')
data = data.merge(albums, how='inner', left_on='album_id', right_on='id')
data = data.merge(performers, how='inner', left_on='performer_id_x', right_on='performer_id')

<ipython-input-90-bed59bcaffd1>:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'name_x'} in the result is deprecated and will raise a MergeError in a future version.
  data = data.merge(albums, how='inner', left_on='album_id', right_on='id')


In [91]:
data['genre'] = data['genre'].replace(' ', '_', regex=True)

### Fill graph

In [92]:
data

,track_id,name_x,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms_x,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,user_id,playcount,song_frequency_per_user,rating,available_markets_x,disc_number,duration_ms_y,explicit,external_ids,external_urls_x,href_x,id_x,is_local,name_y,popularity,preview_url,track_number,type_x,uri_x,album_id,performer_id_x,external_urls_y,href_y,id_y,name_x,type_y,uri_y,album_type,available_markets_y,external_urls,href,id,images,name_y,release_date,release_date_precision,total_tracks,type,uri,performer_id_y,performer_id,artist_ids
0,TRHNGJL128F9310B69,East St. Louis Toodle-Oo,Steely Dan,https://p.scdn.co/mp3-preview/d5e264edddf69878...,0M5U3BpuVd2MscKJnytbFu,"jazz, classic_rock, instrumental, progressive_...",Rock,1985,169453,0.654,0.283,0,-13.873,0,0.0372,0.77100,0.153000,0.3290,0.641,153.745,4,ffffcfeb0c1b66bd212ea58d918c7dc62fb9c3a5,1.0,0.012821,1.0,[],1,169453,False,{'isrc': 'USMC17447200'},{'spotify': 'https://open.spotify.com/track/0M...,https://api.spotify.com/v1/tracks/0M5U3BpuVd2M...,0M5U3BpuVd2MscKJnytbFu,False,East St. Louis Toodle-Oo,0,NaN,11,track,spotify:track:0M5U3BpuVd2MscKJnytbFu,0idh34HVihjNidmDoFiL1B,20f07104-cdde-4658-8b18-1f95c04e47a7,{'spotify': 'https://open.spotify.com/artist/6...,https://api.spotify.com/v1/artists/6P7H3ai06vU...,6P7H3ai06vU1sGvdpBwDmE,Steely Dan,artist,spotify:artist:6P7H3ai06vU1sGvdpBwDmE,compilation,[],{'spotify': 'https://open.spotify.com/album/0i...,https://api.spotify.com/v1/albums/0idh34HVihjN...,0idh34HVihjNidmDoFiL1B,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",A Decade Of Steely Dan (Remastered),1985,year,14,album,spotify:album:0idh34HVihjNidmDoFiL1B,20f07104-cdde-4658-8b18-1f95c04e47a7,20f07104-cdde-4658-8b18-1f95c04e47a7,['6P7H3ai06vU1sGvdpBwDmE']
1,TRPHUBJ128F42B9959,Kid Charlemagne,Steely Dan,https://p.scdn.co/mp3-preview/f9bf942092fa21bf...,00Izb7CLw46eULmD6j5ews,"rock, classic_rock, progressive_rock, 70s",NaN,1985,279306,0.537,0.665,0,-10.674,1,0.0580,0.24600,0.009840,0.6250,0.772,96.644,4,ffc3aa03fb03a2a2d1c034b659f7f0a02302744e,1.0,0.007752,1.0,[],1,279306,False,{'isrc': 'USMC17547218'},{'spotify': 'https://open.spotify.com/track/00...,https://api.spotify.com/v1/tracks/00Izb7CLw46e...,00Izb7CLw46eULmD6j5ews,False,Kid Charlemagne,0,NaN,12,track,spotify:track:00Izb7CLw46eULmD6j5ews,0idh34HVihjNidmDoFiL1B,20f07104-cdde-4658-8b18-1f95c04e47a7,{'spotify': 'https://open.spotify.com/artist/6...,https://api.spotify.com/v1/artists/6P7H3ai06vU...,6P7H3ai06vU1sGvdpBwDmE,Steely Dan,artist,spotify:artist:6P7H3ai06vU1sGvdpBwDmE,compilation,[],{'spotify': 'https://open.spotify.com/album/0i...,https://api.spotify.com/v1/albums/0idh34HVihjN...,0idh34HVihjNidmDoFiL1B,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",A Decade Of Steely Dan (Remastered),1985,year,14,album,spotify:album:0idh34HVihjNidmDoFiL1B,20f07104-cdde-4658-8b18-1f95c04e47a7,20f07104-cdde-4658-8b18-1f95c04e47a7,['6P7H3ai06vU1sGvdpBwDmE']
2,TRIEWNB128E078DC7A,Only A Fool Would Say That,Steely Dan,https://p.scdn.co/mp3-preview/a6172fc4ef630cb5...,0OfxqXHLhWBAhUsHm8zv64,"rock, jazz, classic_rock, 70s",Rock,2000,177600,0.802,0.568,0,-10.926,1,0.0339,0.68700,0.000000,0.1750,0.964,130.046,4,ffffcfeb0c1b66bd212ea58d918c7dc62fb9c3a5,1.0,0.012821,1.0,[],1,177600,False,{'isrc': 'USMC17347183'},{'spotify': 'https://open.spotify.com/track/0O...,https://api.spotify.com/v1/tracks/0OfxqXHLhWBA...,0OfxqXHLhWBAhUsHm8zv64,False,Only A Fool Would Say That,0,NaN,4,track,spotify:track:0OfxqXHLhWBAhUsHm8zv64,5q4bpOVYtyU7Iwlo7r9G4D,20f07104-cdde-4658-8b18-1f95c04e47a7,{'spotify': 'https://open.spotify.com/artist/6...,https://api.spotify.com/v1/artists/6P7H3ai06vU...,6P7H3ai06vU1sGvdpBwDmE,Steely Dan,artist,spotify:artist:6P7H3ai06vU1sGvdpBwDmE,compilation,[],{'spotify': 'https://open.spotify.com/album/5q...,https://api.spotify.com/v1/albums/5q4bpOVYtyU7...,5q4bpOVYtyU7Iwlo7r9G4D,"[{'height': 640, 'url': 'https://i.scdn.co/

In [93]:
# country_dp
# label_dp

In [94]:
for index, row in data.iterrows():
    track_individual = URIRef("http://www.semanticweb.org/music_ontology#track_"+row['track_id'])
    g.add((track_individual, RDF.type, Track_class))
    g.add((track_individual, name_dp, Literal(row['name_x'])))
    g.add((track_individual, popularity_dp, Literal(row['popularity'])))
    g.add((track_individual, acousticness_dp, Literal(row['acousticness'])))
    g.add((track_individual, danceability_dp, Literal(row['danceability'])))
    g.add((track_individual, duration_dp, Literal(row['duration_ms_x'])))
    g.add((track_individual, energy_dp, Literal(row['energy'])))
    g.add((track_individual, instrumentalness_dp, Literal(row['instrumentalness'])))
    g.add((track_individual, liveness_dp, Literal(row['liveness'])))
    g.add((track_individual, loudness_dp, Literal(row['loudness'])))
    g.add((track_individual, speechiness_dp, Literal(row['speechiness'])))
    g.add((track_individual, tempo_dp, Literal(row['tempo'])))
    g.add((track_individual, valence_dp, Literal(row['valence'])))
    g.add((track_individual, releaseYear_dp, Literal(row['year'])))

    performer_individual = URIRef("http://www.semanticweb.org/music_ontology#performer_"+row['performer_id_x'])
    g.add((performer_individual, RDF.type, Performer_class))
    g.add((performer_individual, name_dp, Literal(row['artist'])))

    album_individual = URIRef("http://www.semanticweb.org/music_ontology#album_"+row['album_id'])
    g.add((album_individual, RDF.type, Album_class) )
    g.add((performer_individual, tracksTotal_dp, Literal(row['total_tracks'])))

    genre_individual = URIRef(f"http://www.semanticweb.org/music_ontology#genre_{row['genre']}")
    g.add((genre_individual, RDF.type, Genre_class))
    g.add((performer_individual, name_dp, Literal(row['genre'])))

    # g.add((performer_individual, hasAssociatedGenre_op, genre_individual))
    g.add((track_individual, hasGenre_op, genre_individual))
    g.add((track_individual, partOf_op, album_individual))
    g.add((track_individual, performedBy_op, performer_individual))

Компетентностные вопросы
1. Какой самый популярный жанр музыки среди выпущенных треков?
2. Какая группа выпустила больше всего альбомов в жанре Рок?
3. Альбом какой группы содержит наибольшее количество песен?
4. Какая группа выпустила больше всего альбомов/песен за 2014 год?
5. Какая группа наиболее интернациональная?

In [ ]:
qres0 = g.query("""SELECT ?y ?x ?i ?o ?t
                 WHERE {
                     ?s rdfs:subClassOf* iip-srm:Step_of_Process .
                     ?i rdf:type ?s .
                     ?i iip-srm:affects ?o .
                     ?i iip-srm:performed_in ?x .
                     ?x iip-srm:year ?y .
                     ?i iip-srm:date ?date .
                     ?t iip-srm:is_observation_for ?x .
                     ?t iip-srm:value "AB" .
                     ?t iip-srm:date ?date_t .
                     FILTER ( ?date > ?date_t)
                 } """)

n = 1
for row in qres0:
 print(str(n)+": %s %s %s %s %s" % row)
 n += 1

In [ ]:
qres1 = g.query("""SELECT ?i ?s
                 WHERE {
                     ?s rdfs:subClassOf* iip-srm:Step_of_Process .
                     ?i rdf:type ?s .
                 } """)

n = 1
for row in qres1:
 print(str(n)+": %s %s" % row)
 n += 1

1: http://www.owl-ontologies.com/ato.owl#machine_spreading http://www.owl-ontologies.com/ato.owl#Changes_of_the_agrochemical_properties
2: http://www.owl-ontologies.com/ato.owl#manual_spreading http://www.owl-ontologies.com/ato.owl#Changes_of_the_agrochemical_properties
3: http://purl.org/iip-srm.owl#distribution_over_soil_surface_2015_1 http://www.owl-ontologies.com/ato.owl#Changes_of_the_agrochemical_properties
4: http://purl.org/iip-srm.owl#distribution_over_soil_surface_2016_2 http://www.owl-ontologies.com/ato.owl#Changes_of_the_agrochemical_properties
5: http://purl.org/iip-srm.owl#distribution_over_soil_surface_2017_1 http://www.owl-ontologies.com/ato.owl#Changes_of_the_agrochemical_properties
6: http://purl.org/iip-srm.owl#distribution_over_soil_surface_2017_3 http://www.owl-ontologies.com/ato.owl#Changes_of_the_agrochemical_properties
7: http://purl.org/iip-srm.owl#distribution_over_soil_surface_2018_2 http://www.owl-ontologies.com/ato.owl#Changes_of_the_agrochemical_properties